In [51]:
import dask.dataframe as dd
import numpy
import dask.array as da
import numpy as np 
import pandas as pd

In [47]:
cosine = dd.read_csv('cosine.csv',delimiter=',')
jaccard = dd.read_csv('jaccard.csv',delimiter=',')
pearson = dd.read_csv('pearson.csv',delimiter=',')
u_data = dd.read_csv('u.data',delimiter='\t')
u_genre = dd.read_csv('u.genre',delimiter='|')
u_genre.columns = ['genre','genreId']
u_occupation = dd.read_csv('u.occupation')
u_occupation.columns = ['occupation']
u_user = dd.read_csv('u.user',delimiter='|')
u_user.columns = ['userId','age','gender','occupation','zipcode']
u_item = dd.read_csv('u.item',delimiter='|',encoding='latin-1')
u_item.columns = ['movieId','movieTitle','releaseDate','videoReleaseDate','IMDbId','unknown','action','adventure','animation','children','comedy','crime','documentary','drama','fantasy','filmNoir','horror','musical','mystery','romance','sciFi','thriller','war','western']

In [48]:

cosine_arr = cosine.to_dask_array(lengths=True)

In [54]:
index=0
for i in cosine_arr:
    index+=1
print(index)
u_data.head()

942


,196,242,3,881250949
0,186,302,3,891717742
1,22,377,1,878887116
2,244,51,2,880606923
3,166,346,1,886397596
4,298,474,4,884182806


In [55]:
def split_data_ml100k(data, num_users, num_items, split_mode='random',
                      test_ratio=0.1):
    """Split the dataset in random mode or seq-aware mode."""
    if split_mode == 'seq-aware':
        train_items, test_items, train_list = {}, {}, []
        for line in data.itertuples():
            u, i, rating, time = line[1], line[2], line[3], line[4]
            train_items.setdefault(u, []).append((u, i, rating, time))
            if u not in test_items or test_items[u][-1] < time:
                test_items[u] = (i, rating, time)
        for u in range(1, num_users + 1):
            train_list.extend(sorted(train_items[u], key=lambda k: k[3]))
        test_data = [(key, *value) for key, value in test_items.items()]
        train_data = [item for item in train_list if item not in test_data]
        train_data = pd.DataFrame(train_data)
        test_data = pd.DataFrame(test_data)
    else:
        mask = [
            True if x == 1 else False
            for x in np.random.uniform(0, 1, (len(data))) < 1 - test_ratio]
        neg_mask = [not x for x in mask]
        train_data, test_data = data[mask], data[neg_mask]
    return train_data, test_data
train_data , test_data = split_data_ml100k(u_data,942,1682,test_ratio=0.2)

ValueError: Item wrong length 99999 instead of 0.